In [1]:
# import
import pandas as pd
import numpy as np
from simple.chart import chartParallel, interactTable, chartProfit
from simple.funcs import vwap
from simple.geneopt import GeneOpt
from multiprocessing import current_process
from simple.backtest import getProfitDict, npBacktestLimit

In [2]:
T = np.load('data/tick.npz')['BTCUSDT'].view(np.recarray)
len(T)

889360

In [3]:
def model(Period: int = (1000, 6000, 500), StdDev: float = (2, 3.6, 0.1)):
    if Period == 0 or StdDev == 0: return {}
    Tick = T.Price
    Center = vwap(T, Period)
    _std = pd.Series(Tick).rolling(Period).std().bfill().values
    qA = Center + _std * StdDev
    qB = Center - _std * StdDev

    # returns profit metrics during gene search
    _trades = npBacktestLimit(T, qA, qB)
    return getProfitDict(_trades) if current_process().daemon else {**locals(), **chartProfit(_trades)}

In [4]:
# Genetic optimizer
G = GeneOpt(model)
G.maximize(population_size=100, generations=5)

  0%|          | 0/5 [00:00<?, ?it/s]

{'Period': 5500, 'StdDev': 3.434231324949654}

In [5]:
# Genetic result browser
X = pd.DataFrame(G.log, columns=G.log_columns).drop_duplicates().sort_values('Profit', ascending=False)

interactTable(model, X, height=600)

In [6]:
chartParallel(X)

FigureWidget({
    'data': [{'dimensions': [{'label': 'index',
                              'range': [0, 491],
                              'values': array([192, 354, 473,  88,  15, 340, 106, 170, 103,  89, 253, 400, 216,  31,
                                                55, 235,  67, 107, 188,  62, 245, 355,  39,  27, 252, 251,  87,  11,
                                               174, 229, 190, 261, 234, 136, 333, 167, 191, 127,  93, 278, 346, 376,
                                                20, 100, 135, 250, 317, 491, 362, 377, 158,  38,  65,  51, 164,  81,
                                               114, 113, 226, 298, 181, 358,  64,  57, 180, 199, 326, 232,  10, 391,
                                                 8, 169, 165,  16, 227, 189, 126, 341, 131, 183, 233,  63, 320,  46,
                                               176, 214,  98,  86,  80, 398,  47, 112, 247,  40, 154, 137,  97, 197,
                                               159,  48,   1, 111,  2